엔터카는 중고차를 매매를 하는 회사이다. 엔터카 데이터 분석가는 중고 자동차에 대한 판매가격, 누적주행거리, 무게 등의 정보를 활용하여 자동차 사양에 따른 판매가격 예측모델을 만들고자 한다. 분석에 사용할 해당 데이터에 관한 정보는 아래와 같다.
1. enter_car_parti.txt : 자동차 사양 중 연속형 테이터 (1,004 Rows, 7 Columns) 
2. enter_car_part2.txt : 자동차 사양 중 범주형 데이터 (1,004 Rows, 4 Columns
각 컬럼의 정보는 다음과 같다. 

[enter_car_part1.txt]
- ID : 자동차ID, (Double)
- Price : 판매가격, (Double)
- Age : 차량 연식, (Double)
- KM : 누적 주행 거리, (Double)
- HP : 마력, (Double)
- CC : 배기량, (Double)
- Weight : 무게, (Double)

[enter_car_part2.txt]
- ID : 자동차ID (Double)
- FuelType : 사용 연료, (String)
- TransmissionType : 변속기 타입, (String)
- Doors : 문 개수, (Double)

In [311]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import urllib
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas.stats.api import ols

## Data read and preprocessing

In [2]:
data_continuous = pd.read_csv('enter_car_part1.txt')
data_discrete = pd.read_csv('enter_car_part2.txt')

In [3]:
print(type(data_continuous), data_continuous.shape)

<class 'pandas.core.frame.DataFrame'> (1004, 7)


In [4]:
print(type(data_discrete), data_discrete.shape)

<class 'pandas.core.frame.DataFrame'> (1004, 4)


#### 결측치 제거

In [5]:
np.sum(pd.isnull(data_continuous)), np.sum(pd.isnull(data_discrete))

(ID        0
 Price     0
 Age       0
 KM        0
 HP        0
 CC        0
 Weight    0
 dtype: int64,
 ID                  0
 FuelType            9
 TransmissionType    0
 Doors               0
 dtype: int64)

In [6]:
# 범주형 데이터에 결측치 제거
data_discrete_del_null = data_discrete.dropna(axis=0)

In [7]:
data_discrete_del_null

,ID,FuelType,TransmissionType,Doors
0,43.0,Petrol,Manual,2doors
1,44.0,Diesel,Manual,2doors
2,45.0,Diesel,Manual,5doors
3,46.0,Diesel,Manual,5doors
4,47.0,Diesel,Manual,5doors
...,...,...,...,...
999,974.0,Petrol,Manual,4doors
1000,984.0,Petrol,Manual,4doors
1001,986.0,Petrol,Automatic,4doors
1002,991.0,Petrol,Manual,4doors


Note: 분석에 앞서 다음 전처리를 수행하고 기본 데이터셋을 만드시오 
1. 데이터 A와 데이터 B를 자동차 ID를 기준으로 하나로 합친다 
2. 데이터에 결측이 존재하는 경우 삭제 
<주의 사항> 브라이틱스 이용자의 경우 결측임에도 공란 ()으로 인식하여 결측으로 여기지 않으므로 이를 발견하여 제거 필요

In [142]:
baseData = pd.merge(data_continuous, data_discrete_del_null, how='inner', on='ID')

In [143]:
baseData

,ID,Price,Age,KM,HP,CC,Weight,FuelType,TransmissionType,Doors
0,1,13500,23,46986,90,2000,1165,Diesel,Manual,4doors
1,2,13750,23,72937,90,2000,1165,Diesel,Manual,3doors
2,3,13950,24,41711,90,2000,1165,Diesel,Manual,4doors
3,4,14950,26,48000,90,2000,1165,Diesel,Manual,3doors
4,5,13750,30,38500,90,2000,1170,Diesel,Manual,4doors
...,...,...,...,...,...,...,...,...,...,...
990,1000,10950,57,40214,86,1300,1025,Petrol,Manual,4doors
991,1001,8950,60,40000,86,1300,1000,Petrol,Manual,4doors
992,1002,8950,66,39222,86,1300,1045,Petrol,Automatic,3doors
993,1003,8250,66,39144,110,1600,1050,Petrol,Manual,2doors


#### Doors 기준 평균 확인

In [144]:
grouped = baseData['Price'].groupby(baseData['Doors'])
grouped.mean()

Doors
2doors    12371.695431
3doors    11422.399061
4doors    11283.077193
5doors    12407.080000
Name: Price, dtype: float64

[문제 2-1]
변속기 타입(TransmissionType)이 판매가격에 영향을 주는지에 대한 통계적 검정을 진행하고자 한다. 적절한 검증 방법을 선택하여 유의수준 10%에서 검증하고 아래의 물음에 답하시오. 귀무가설 기각여부를 Y/N으로 기술, 분산 동질성 여부를 Y/N으로 기술하고 마지막으로 검정 통계량이 발생할 확률을 소수점 셋째 자리까지 작성하 시오(소수점 넷째 자리 미만은 버림)

In [207]:
baseData.TransmissionType.unique()

array(['Manual', 'Automatic'], dtype=object)

In [208]:
sampleListTransmissioType = [0 if i == 'Manual' else 1 for i in baseData.TransmissionType]

gManual = [baseData.Price[i] for i in range(len(sampleListTransmissioType)) if sampleListTransmissioType[i] == 0]
gAutomatic = [baseData.Price[i] for i in range(len(sampleListTransmissioType)) if sampleListTransmissioType[i] == 1]

In [210]:
np.array(gManual).shape, np.array(gAutomatic).shape

((946,), (49,))

In [220]:
result = stats.ttest_ind(np.array(gAutomatic), np.array(gManual), equal_var=True)

#### 결과
유의 수준 10%에서 귀무가설을 기각한다.

In [221]:
result

Ttest_indResult(statistic=1.6694782121803504, pvalue=0.09533783345926539)

[문제2-2]
문 개수가 판매가격에 영향을 주는지에 대한 통계적 검정을 진행하고자 한다. 적절한 검증 방법을 선택하여 유의수준 5%에서 검증하시오.

In [150]:
baseData.Doors.unique()

array(['4doors', '3doors', '2doors', '5doors'], dtype=object)

In [197]:
sample_list2 = [2 if i == '2doors' else -1 for i in baseData.Doors]
sample_list3 = [3 if i == '3doors' else -1 for i in baseData.Doors]
sample_list4 = [4 if i == '4doors' else -1 for i in baseData.Doors]
sample_list5 = [5 if i == '5doors' else -1 for i in baseData.Doors]

In [205]:
sampling2 = [baseData.Price[i] for i in range(len(sample_list2)) if sample_list2[i]>0]
sampling3 = [baseData.Price[i] for i in range(len(sample_list3)) if sample_list3[i]>0]
sampling4 = [baseData.Price[i] for i in range(len(sample_list4)) if sample_list4[i]>0]
sampling5 = [baseData.Price[i] for i in range(len(sample_list5)) if sample_list5[i]>0]

#### 결과
F 검정 결과 귀무가설 기각 확인
- 문에 갯수에 따라서 각경이 다르다

In [206]:
stats.f_oneway(np.array(sampling2), np.array(sampling3), np.array(sampling4), np.array(sampling5))

F_onewayResult(statistic=6.6527610698417705, pvalue=0.0001894354232972141)

[문제2-3] Linear Regression(선형 회귀 분석)을 통해 차량 판매가격을 예측하는 모델을 만들고자 한다. Price(가격)을 종속변수로 하고, 연속형 변수에 대해서는 Price와의 Pearson 상관계수의 절대값이 0.3이상인 것들을 독립변수로 사용하고 범주형 데이터는 모두 변수를 Dummy 변수화하여 독립변수로 사용한다. 

기존변수의 고유한 개수(k)에서 하나를 뺀 개수만큼(k-1)의 Dummy 변수를 만드시오 

모델에 들어간 변수들 중 유의수준 1%에서 통계적으로 유의한 변수의 개수(단 상수항은 제외)와 Adjusted R2(조정 결정계수) 값을 차례로 서술하시오

#### 피어슨 상관관계

In [237]:
print(baseData[['Price', 'Age', 'KM', 'HP', 'CC', 'Weight']].corr(method='pearson'))

           Price       Age        KM        HP        CC    Weight
Price   1.000000 -0.868557 -0.593607  0.360227  0.187852  0.614633
Age    -0.868557  1.000000  0.549199 -0.176203 -0.148005 -0.508140
KM     -0.593607  0.549199  1.000000 -0.347763  0.294350 -0.075802
HP      0.360227 -0.176203 -0.347763  1.000000  0.021706  0.089929
CC      0.187852 -0.148005  0.294350  0.021706  1.000000  0.627700
Weight  0.614633 -0.508140 -0.075802  0.089929  0.627700  1.000000


#### Dummy Data 변환 - 원핫 인코딩 으로 변환

In [256]:
## 범주형 자료 Dummy로 변환 - , 'FuelType', 'TransmissionType', 'Doors'
enc = OneHotEncoder()

In [257]:
type(baseData), baseData.FuelType.unique()

(pandas.core.frame.DataFrame,
 array(['Diesel', 'Petrol', 'CNG'], dtype=object),
 995)

In [295]:
tmp = []
for x in baseData.FuelType:
    if x == 'CNG':
        tmp.append(0)
    elif x == 'Diesel':
        tmp.append(1)
    else:
        tmp.append(2)
tmp = np.array(tmp).reshape(-1, 1)

enc.fit(tmp)
oneHot = enc.transform(tmp).toarray()
df1 = pd.DataFrame(oneHot, columns=['FuelType_CNG', 'FuelType_Diesel', 'FuelType_Petrol'])

In [269]:
baseData.TransmissionType.unique()

(pandas.core.frame.DataFrame, array(['Manual', 'Automatic'], dtype=object))

In [288]:
tmp = [0 if x == 'Automatic' else 1 for x in baseData.TransmissionType]
tmp = np.array(tmp).reshape(-1, 1)
tmp.shape

enc.fit(tmp)
oneHot = enc.transform(tmp).toarray()
df2 = pd.DataFrame(oneHot, columns=['TransmissionType_Automatic', 'TransmissionType_Manual'])

(995, 1)

In [281]:
baseData.Doors.unique()

array(['4doors', '3doors', '2doors', '5doors'], dtype=object)

In [294]:
tmp = []
for x in baseData.Doors:
    if x == '2doors':
        tmp.append(0)
    elif x == '3doors':
        tmp.append(1)
    elif x == '4doors':
        tmp.append(2)
    else:
        tmp.append(3)
tmp = np.array(tmp).reshape(-1, 1)

enc.fit(tmp)
oneHot = enc.transform(tmp).toarray()
df3 = pd.DataFrame(oneHot, columns=['Doors_2Doors', 'Doors_3Doors', 'Doors_4Doors', 'Doors_5Doors'])

In [320]:
baseData1 = pd.concat([baseData, df1], axis=1)
baseData2 = pd.concat([baseData1, df2], axis=1)
baseData3 = pd.concat([baseData2, df3], axis=1)
baseData3

,ID,Price,Age,KM,HP,CC,Weight,FuelType,TransmissionType,Doors,FuelType_CNG,FuelType_Diesel,FuelType_Petrol,TransmissionType_Automatic,TransmissionType_Manual,Doors_2Doors,Doors_3Doors,Doors_4Doors,Doors_5Doors
0,1,13500,23,46986,90,2000,1165,Diesel,Manual,4doors,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2,13750,23,72937,90,2000,1165,Diesel,Manual,3doors,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,3,13950,24,41711,90,2000,1165,Diesel,Manual,4doors,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,4,14950,26,48000,90,2000,1165,Diesel,Manual,3doors,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,5,13750,30,38500,90,2000,1170,Diesel,Manual,4doors,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,1000,10950,57,40214,86,1300,1025,Petrol,Manual,4doors,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
991,1001,8950,60,40000,86,1300,1000,Petrol,Manual,4doors,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
992,1002,8950,66,39222,86,1300,1045,Petrol,Automatic,3doors,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
993,1003,8250,66,39144,110,1600,1050,Petrol,Manual,2doors,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [326]:
result = ols(formula='Price ~ Age + KM + HP + Weight + FuelType_CNG + FuelType_Diesel + FuelType_Petrol + TransmissionType_Automatic + TransmissionType_Manual + Doors_2Doors + Doors_3Doors + Doors_4Doors + Doors_5Doors', data=baseData3).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     612.5
Date:                Sun, 06 Sep 2020   Prob (F-statistic):               0.00
Time:                        11:16:53   Log-Likelihood:                -8617.7
No. Observations:                 995   AIC:                         1.726e+04
Df Residuals:                     984   BIC:                         1.731e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [328]:
result = smf.ols(formula='Price ~ Age + KM + HP + Weight + FuelType_CNG + FuelType_Diesel + TransmissionType_Automatic + TransmissionType_Manual + Doors_2Doors + Doors_3Doors + Doors_4Doors', data=baseData3).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     612.5
Date:                Sun, 06 Sep 2020   Prob (F-statistic):               0.00
Time:                        11:17:17   Log-Likelihood:                -8617.7
No. Observations:                 995   AIC:                         1.726e+04
Df Residuals:                     984   BIC:                         1.731e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

[문제2-4]문제 3에서 생성된 모델을 이용하여 차량 연식(Age) 7년 누적주행거리(KM) 12,500KM, 115마력(HP) 배기량(CC) 3000CC, 무게(Weight) 1230Kg, 사용연료(FuelType) Diesel, 변속기 타입(TransmissionType) Automatic, 문 개수(Doors) 4doors인 차량의 판매각겨을 예측하시오

In [332]:
x = baseData3[['Age', 'KM', 'HP', 'Weight', 'FuelType_CNG', 'FuelType_Diesel', 'FuelType_Petrol', 'TransmissionType_Automatic', 'TransmissionType_Manual', 'Doors_2Doors', 'Doors_3Doors', 'Doors_4Doors', 'Doors_5Doors']]
y = baseData3['Price']

model = LinearRegression(fit_intercept=True)
model.fit(x, y) 

LinearRegression()

In [333]:
preData = [[7, 12500, 115, 1230, 0, 1, 0, 1, 0, 0, 0, 1, 0]]
my_predict = model.predict(preData)
my_predict

array([21599.4406351])